In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
#from xgboost import XGBRegressor
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

train = pd.read_csv('C:/Users/Yunfei Cai/Desktop/test_Sample data (3d)(2).csv')


#初始設定----------------------------------------------------------------------
train.dropna(inplace=True)


train_y=train['Event count']

#all_data = pd.concat([train, test], ignore_index = True)
train.drop(['COPLNT_DAY'],axis=1)

train_x=train
train_x.drop(columns=["Event count"])


train_x.dropna(axis=0, how='any')
train_x = pd.get_dummies(train_x)

#seperate the dataset----------------------------------------------------------
#case 1




#training set
train_x=train_x.values
train_y=train_y.values
train_x=train_x[:,1:8]
train_y=train_y.reshape(-1,1)



# 標準化
scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
# test = scaler.transform(test)   

#PCA---------------------------------------------------------------------------

pca_num=0
delta=0.1

pca=PCA(n_components = 0.999999)
train_x=pca.fit_transform(train_x)
 
print(train_x.shape)
dimension=train_x.shape[1]
print(train_y)

C:\Users\Yunfei Cai\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


(23646, 7)
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [2]:
#SVM
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC

# mb = MultiLabelBinarizer()
# train_y = mb.fit_transform(train_y)
# print(train_y)
K=5
kf = KFold(n_splits=K)
model = OneVsRestClassifier(svm.SVC(kernel='linear'))

# train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.3,random_state=0) 
accuracy_his=[]
for train_index, test_index in kf.split(train_x):
    y_model=OneVsRestClassifier(LinearSVC(random_state=0)).fit(train_x[train_index],train_y[train_index]).predict(train_x[test_index])

#     model.fit(train_x[train_index],train_y[train_index])
#     y_model = model.predict (train_x[test_index])
    accuracy=sum(y_model==train_y[test_index])/len(train_y[test_index])
    accuracy_his=np.append(accuracy_his,accuracy )
    
accuracy_final=np.mean(accuracy)
print(accuracy_final)
# print(accuracy)
# print("\n")
#print(y_model)
# print(train_y[test_index])


#     mse=np.mean((np.round(y_model) != train_y[test_index])**2)
#     rmse+=np.sqrt(mse)/K
# print(rmse)

# y_model_test = model.predict (test_x) 
# print("finish, plot the scatter figure")
# plt.figure(3)
# fig, ax = plt.subplots()
# ax.scatter(y_model_test, test_y)
# ax.plot([test_y.min(), test_y.max()], [test_y.min(), test_y.max()], 'k--', lw=3)
# ax.set_xlabel('Measured')
# ax.set_ylabel('Predicted')


#plt.show()   
#plt.pause(0.1)

0.8984986255022205


In [3]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics

rf = RandomForestClassifier(oob_score=True, random_state=10)
rf.fit(train_x,train_y)
print(rf.oob_score_)
y_rf = rf.predict (train_x)
y_rf=y_rf.reshape(-1,1)
print(y_rf)

print(train_x.shape)
print(train_y.shape)

tuned_parameters = [{'min_samples_leaf':[1,2,3,4], 'n_estimators':[10,50,100,150,200]}]
rfc = RandomForestClassifier(max_depth=2, random_state=0)
clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5, n_jobs=1)
clf.fit(train_x,train_y) 
print('Best parameters:') 
print(clf.best_params_)

0.9684513236911105
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
(23646, 7)
(23646, 1)


IndexError: too many indices for array